In [1]:
import numpy as np
import pandas as pd


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/diabetes.csv')

In [4]:
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [9]:
X= scaler.fit_transform(X)

In [11]:
X.shape

(768, 8)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.1, random_state=42)

In [73]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [16]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics=['Accuracy'])

In [17]:
model.fit(X_train, y_train, batch_size=32,validation_data=(X_test, y_test), epochs=100)

Epoch 1/100
22/22 [==============================] - 1s 15ms/step - loss: 0.7814 - Accuracy: 0.3936 - val_loss: 0.7647 - val_Accuracy: 0.4026
Epoch 2/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6830 - Accuracy: 0.5673 - val_loss: 0.6923 - val_Accuracy: 0.6104
Epoch 3/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6205 - Accuracy: 0.7019 - val_loss: 0.6398 - val_Accuracy: 0.6364
Epoch 4/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5799 - Accuracy: 0.7467 - val_loss: 0.6072 - val_Accuracy: 0.6623
Epoch 5/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5524 - Accuracy: 0.7569 - val_loss: 0.5836 - val_Accuracy: 0.6623
Epoch 6/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5319 - Accuracy: 0.7699 - val_loss: 0.5652 - val_Accuracy: 0.6753
Epoch 7/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5156 - Accuracy: 0.7656 - val_loss: 0.5519 - val_Accuracy: 0.6883
Epoch 8/100


How to Automate to find the best suitable parameter

- 1. How to select the appropriate optimizer
- 2. No.of Nodes in a layer
- 3. how to select no.of layers
-4. All in all one model

In [18]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.7 MB/s eta 0:00:00


In [19]:
import kerastuner as kt

<ipython-input-19-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


1. How to find the appropriate Optimizer

In [32]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values= ['Adam','SGD','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss ='binary_crossentropy', metrics=['accuracy'])

  return model

In [33]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [34]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

In [36]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'SGD'}

In [37]:
model = tuner.get_best_models(num_models=1)[0]

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100,initial_epoch=6)

Epoch 7/100
22/22 [==============================] - 0s 7ms/step - loss: 0.5525 - accuracy: 0.7395 - val_loss: 0.5823 - val_accuracy: 0.7403
Epoch 8/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5452 - accuracy: 0.7410 - val_loss: 0.5776 - val_accuracy: 0.7273
Epoch 9/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5386 - accuracy: 0.7410 - val_loss: 0.5736 - val_accuracy: 0.7403
Epoch 10/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5327 - accuracy: 0.7366 - val_loss: 0.5700 - val_accuracy: 0.7403
Epoch 11/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5274 - accuracy: 0.7381 - val_loss: 0.5663 - val_accuracy: 0.7403
Epoch 12/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5226 - accuracy: 0.7381 - val_loss: 0.5634 - val_accuracy: 0.7403
Epoch 13/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5183 - accuracy: 0.7438 - val_loss: 0.5609 - val_accuracy: 0.7403
Epoch 14/

2. No.of Nodes in a layer

In [46]:
def build_model(hp):

  model = Sequential()

  units = hp.Int('units', min_value=8, max_value = 128, step=8)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [47]:
tuner = kt. RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name='units'
)

In [48]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 10s


In [49]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [50]:
model = tuner.get_best_models(num_models=1)[0]

In [51]:
model.fit(X_train,y_train, batch_size= 32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
22/22 [==============================] - 1s 13ms/step - loss: 0.6463 - accuracy: 0.6802 - val_loss: 0.6212 - val_accuracy: 0.7532
Epoch 8/100
22/22 [==============================] - 0s 5ms/step - loss: 0.6263 - accuracy: 0.7091 - val_loss: 0.6072 - val_accuracy: 0.7403
Epoch 9/100
22/22 [==============================] - 0s 5ms/step - loss: 0.6095 - accuracy: 0.7192 - val_loss: 0.5951 - val_accuracy: 0.7273
Epoch 10/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5948 - accuracy: 0.7279 - val_loss: 0.5846 - val_accuracy: 0.7143
Epoch 11/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5818 - accuracy: 0.7410 - val_loss: 0.5755 - val_accuracy: 0.7273
Epoch 12/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5703 - accuracy: 0.7511 - val_loss: 0.5675 - val_accuracy: 0.7143
Epoch 13/100
22/22 [==============================] - 0s 5ms/step - loss: 0.5603 - accuracy: 0.7540 - val_loss: 0.5606 - val_accuracy: 0.7143
Epoch 14

3. How to select Number of Hidden layers

In [53]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(72, activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

  return model



In [54]:
tuner = kt.RandomSearch(
    build_model,
    objective=['val_accuracy'],
    max_trials=3,
    directory='mydir',
    project_name='num_layers'
)

In [55]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 03s]
multi_objective: -0.649350643157959

Best multi_objective So Far: -0.6623376607894897
Total elapsed time: 00h 00m 10s


In [56]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [57]:
model = tuner.get_best_models(num_models=1)[0]

In [58]:
model.fit(X_train,y_train, batch_size= 32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
22/22 [==============================] - 2s 15ms/step - loss: 0.6825 - accuracy: 0.6469 - val_loss: 0.6786 - val_accuracy: 0.6494
Epoch 8/100
22/22 [==============================] - 0s 7ms/step - loss: 0.6740 - accuracy: 0.6512 - val_loss: 0.6729 - val_accuracy: 0.6494
Epoch 9/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6676 - accuracy: 0.6512 - val_loss: 0.6685 - val_accuracy: 0.6494
Epoch 10/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6625 - accuracy: 0.6512 - val_loss: 0.6651 - val_accuracy: 0.6494
Epoch 11/100
22/22 [==============================] - 0s 7ms/step - loss: 0.6583 - accuracy: 0.6512 - val_loss: 0.6621 - val_accuracy: 0.6494
Epoch 12/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6548 - accuracy: 0.6512 - val_loss: 0.6596 - val_accuracy: 0.6494
Epoch 13/100
22/22 [==============================] - 0s 6ms/step - loss: 0.6517 - accuracy: 0.6512 - val_loss: 0.6573 - val_accuracy: 0.6494
Epoch 14

4. All in all

In [85]:
def build_model(hp):

  model = Sequential()

  counter=0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:

      model.add(Dense(
          hp.Int('Units'+ str(i), min_value=8, max_value=128, step=8),
          activation=hp.Choice('activation'+str(i), values=['relu','tanh', 'sigmoid']),
          input_dim=8
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(
          hp.Int('Units'+ str(i), min_value=8, max_value=128, step=8),
          activation=hp.Choice('activation'+str(i), values=['relu','tanh', 'sigmoid'])
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1, activation='sigmoid'))

  optimizer= hp.Choice('activation', values=['adam','sgd','nadam','rmsprop','adadelta'])


  model.compile(optimizer= optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [86]:
tuner = kt.RandomSearch(build_model,
                        objective=['val_accuracy'],
                        max_trials=3,
                        directory='mydir',
                        project_name='All2')

In [87]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 06s]
multi_objective: -0.649350643157959

Best multi_objective So Far: -0.649350643157959
Total elapsed time: 00h 00m 15s


In [88]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'Units0': 24,
 'activation0': 'tanh',
 'dropout0': 0.7,
 'activation': 'sgd',
 'Units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'Units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'Units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'Units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'Units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'Units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'Units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [89]:
model = tuner.get_best_models(num_models=1)[0]

In [90]:
model.fit(X_train,y_train, epochs=200, initial_epoch=5, validation_data=(X_test,y_test))

Epoch 6/200
22/22 [==============================] - 2s 17ms/step - loss: 0.6742 - accuracy: 0.6614 - val_loss: 0.6724 - val_accuracy: 0.6494
Epoch 7/200
22/22 [==============================] - 0s 7ms/step - loss: 0.6732 - accuracy: 0.6440 - val_loss: 0.6683 - val_accuracy: 0.6494
Epoch 8/200
22/22 [==============================] - 0s 8ms/step - loss: 0.6649 - accuracy: 0.6556 - val_loss: 0.6648 - val_accuracy: 0.6494
Epoch 9/200
22/22 [==============================] - 0s 8ms/step - loss: 0.6610 - accuracy: 0.6512 - val_loss: 0.6619 - val_accuracy: 0.6494
Epoch 10/200
22/22 [==============================] - 0s 11ms/step - loss: 0.6590 - accuracy: 0.6512 - val_loss: 0.6594 - val_accuracy: 0.6494
Epoch 11/200
22/22 [==============================] - 0s 11ms/step - loss: 0.6561 - accuracy: 0.6512 - val_loss: 0.6573 - val_accuracy: 0.6494
Epoch 12/200
22/22 [==============================] - 0s 12ms/step - loss: 0.6542 - accuracy: 0.6483 - val_loss: 0.6555 - val_accuracy: 0.6494
Epoch 